In [21]:
import sys
import json
aggregation = {'distance':1000}
constant = {'cst_incline':1}
road_weight = {'residential':0.5,
            'secondary_link':0.3,
            'secondary':0.3,
            'tertiary':0.3,
            'cycleway':1.1,
            'primary':0.3,
            'motorway_link':0,
            'primary_link':0.3,
            'tertiary_link':0.3,
            'motorway':0,
            'trunk':0.2, 
            'trunk_link':0.2  
            }
shared_cycleway_weight = {'residential':0.8,
            'secondary_link':0.5,
            'secondary':0.5,
            'tertiary':0.5,
            'cycleway':1.1,
            'primary':0.5,
            'motorway_link':0,
            'primary_link':0.5,
            'tertiary_link':0.5,
            'motorway':0,
            'trunk':0.4, 
            'trunk_link':0.4   
        }

params = {'aggregation':aggregation,'constant':constant, 'road_weight': road_weight, 'shared_cycleway_weight': shared_cycleway_weight}

         

default = {'training_folder': '../../scenarios/NTA Dublin M50', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/NTA Dublin M50', 'params': {'aggregation': {'distance': 1000}, 'constant': {'cst_incline': 1}, 'road_weight': {'residential': 0.5, 'secondary_link': 0.3, 'secondary': 0.3, 'tertiary': 0.3, 'cycleway': 1.1, 'primary': 0.3, 'motorway_link': 0, 'primary_link': 0.3, 'tertiary_link': 0.3, 'motorway': 0, 'trunk': 0.2, 'trunk_link': 0.2}, 'shared_cycleway_weight': {'residential': 0.8, 'secondary_link': 0.5, 'secondary': 0.5, 'tertiary': 0.5, 'cycleway': 1.1, 'primary': 0.5, 'motorway_link': 0, 'primary_link': 0.5, 'tertiary_link': 0.5, 'motorway': 0, 'trunk': 0.4, 'trunk_link': 0.4}}}


In [22]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from quetzal.engine.road_model import RoadModel, _reverse_geom
from quetzal.engine.pathfinder_utils import get_path, parallel_dijkstra, build_index, sparse_matrix
from quetzal.engine.msa_utils import get_zone_index, assign_volume
import numpy as np
import random
import matplotlib.pyplot as plt
from shapely.geometry import Point
from typing import Tuple
from geopy.distance import geodesic  # works for geopy version >=2
#from sklearn.cluster import KMeans
from syspy.spatial.spatial import nearest, agglomerative_clustering, voronoi_diagram_dataframes, add_geometry_coordinates
from quetzal.engine.pathfinder_utils import simple_routing,get_path
num_cores = 1

In [23]:
def drop_duplicated_links(links: gpd.GeoDataFrame,
                          sort_column: str = 'maxspeed', 
                          ascending: bool = False, 
                          return_dropped:bool = False)-> Tuple[gpd.GeoDataFrame,list]:
    '''
    drop duplicated links (a,b) with condition sort_column. if maxspeed and ascending=False, keep faster road
    '''
    before = set(links.index)
    links['dup'] = links['a'] + links['b']
    links = links.sort_values(sort_column, ascending=ascending).drop_duplicates('dup').sort_index()
    links = links.drop(columns='dup')
    after = set(links.index)
    diff = list(before-after)
    print(len(diff), 'links dropped')
    return links, diff

def get_epsg(lat: float, lon: float) -> int:
    '''
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

def zones_nearest_node(zones,nodes,drop_duplicates=False):
    # getting zones centroids
    centroid = zones.copy()
    centroid['geometry'] = centroid.centroid
    # finding nearest node
    neigh = nearest(centroid, nodes, n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    centroid['node_index'] = centroid.index.map(zone_node_dict.get)
    print('max_distance found: ', neigh['distance'].max())
    # check for duplicated nodes. if there is. drop the duplicated zones.
    if drop_duplicates:
        if len(centroid.drop_duplicates('node_index')) != len(centroid):
            print('there is zones associates to the same road_node')
            # duplicated = centroid[centroid['node_index'].duplicated()]['node_index'].values
            print('dropping zones: ')
            print(centroid[centroid['node_index'].duplicated()].index.values)
            centroid = centroid.drop_duplicates('node_index')
    return centroid

def get_flight_distance(x):
    # inputs : [(lat,lon), (lat,lon)]. or [(y,x),(y,x)]
    # however. geodesic use lon,lat. so its inverted here

    
    return geodesic(x[0], x[1]).m

In [24]:
#visualization.py
import six
import unicodedata
import matplotlib.pyplot as plt

def rounding(x):
    if type(x)==str:
        return x
    elif x<10:
        return float(np.round(x,2))
    elif x<100:
        return float(np.round(x,1))
    else:
        return int(x)

def put_units_in_row(df):
    df = df.applymap(rounding, na_action='ignore')
    index = df.index.values
    df.loc['units'] = [col.split(' ')[1] for col in df.columns]
    df.columns = [col.split(' ')[0] for col in df.columns]
    index = np.insert(index,0,'units')
    df = df.loc[index]
    return df

def normalize(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode().replace(' ', '_').lower().replace(',', '')

class Current:
    def __init__(self):
        self.s = ''
        
    def replace_seen(self, s):
        if self.s == s:
            to_return = ''
        else :
            to_return = s 
        self.s = s
        return to_return

def label_index(data):
    ix_name = data.index.name
    c_name = data.columns.name 
    t = data.copy()
    if c_name is not None:
        t.index.name = c_name
    if ix_name is not None:
        ix = list(data.index)
        t = t.reindex([ix_name] + list(ix))
        t = t.fillna('')
    return t

def render_mpl_table(
    data, 
    col_width=3.0, 
    row_height=0.625, 
    font_size=14,
    header_size=14,
    index_width_ratio=2,
    header_color='#9d1a1e', 
    header_font_color = 'w',
    sub_header_color='#d22328',
    row_colors=['#f1f1f2', 'w'], 
    edge_color='w',
    index_edge_color='#9d1a1e',
    bbox=[0, 0, 1, 1], 
    header_columns=0,
    figsize=None,
    ax=None, 
    dpi=96,
    **kwargs
):
    #c_levels = len(data.columns.names)
    #c_first = data.columns.names[0]
    
    
    i_levels = len(data.index.names)
    i_first = list(data.index.names)[0]

    data = data.reset_index()
    current = Current()
    try:
        data[i_first] = data[i_first].apply(lambda s: current.replace_seen(s))
    except KeyError:
        pass

    
    if figsize:
        col_width = figsize[0] / (len(data.T) + (index_width_ratio - 1))
        row_height = figsize[1] / (len(data) +1)
        
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size, dpi=dpi)
        ax.axis('off')
    
    

    mpl_table = ax.table(
        cellText=data.values, 
        bbox=bbox, 
        colLabels=data.columns, 
        colWidths= [col_width * index_width_ratio ] + [col_width for c in data.columns[1:]],
        **kwargs
    )

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)
    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(index_edge_color)
        if k[0] < 1 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color=header_font_color)

            cell.set_text_props( color=header_font_color)
            cell.set_fontsize(header_size)
            cell.set_facecolor(header_color)

        elif k[1] < i_levels:
            cell.set_text_props(weight='bold', color=header_font_color)
            cell.set_text_props( color=header_font_color)
            cell.set_fontsize(header_size)
            cell.set_facecolor(sub_header_color)
        else:
            cell.set_edgecolor(edge_color)
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax


In [25]:
base_folder = argv['training_folder']
road_folder = base_folder + '/inputs/road/'
demand_folder = base_folder +'/inputs/'
od_folder = base_folder + '/inputs/od/'
output_folder = base_folder +'/outputs/'
print(road_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


../../scenarios/NTA Dublin M50/inputs/road/


# inputs

In [26]:
cst_incline = argv['params' ]['constant']['cst_incline']
cst_road = argv['params']['road_weight']
cst_shared = argv['params']['shared_cycleway_weight']

In [27]:
#make sure its float
cst_incline = float(cst_incline)
cst_road = {k:float(v) for k,v in cst_road.items()}
cst_shared = {k:float(v) for k,v in cst_shared.items()}

In [28]:
links = gpd.read_file(road_folder + 'road_links.geojson') 
nodes = gpd.read_file(road_folder + 'road_nodes.geojson')
links = links.set_index('index')
nodes = nodes.set_index('index')

In [110]:
def get_filepath(path,filename):
    '''
    get filePath cas insensitive (ex: read demand.csv but file is name DEMAND.csv)
    path:'path/ ex: '../../scenarios/base/inputs/'
    filename: ex: demand.csv
    '''
    files = os.listdir(demand_folder)
    file = [file for file in files if filename.lower() == file.lower()]
    if len(file)==0:
        raise Exception(f'{path+filename} does not exist')
    return path+file[0]

In [111]:
demand = pd.read_csv(get_filepath(demand_folder ,'demand.csv'))
assert all([col in demand.columns for col in ['lon_ori','lat_ori','lon_des','lat_des','volume'] ]), 'need lon_ori, lat_ori, lon_des, lat_des, volume in demand'
demand = demand[['lon_ori','lat_ori','lon_des','lat_des','volume']]

In [112]:
od_file = od_folder + 'od.geojson'
od_file_provided = os.path.isfile(od_file)
if od_file_provided:
    od_test = gpd.read_file(od_folder + 'od.geojson')
    if 'name' not in od_test.columns:
        od_test['name'] = od_test['index']
    od_test['name'] = od_test['name'].fillna(od_test['index'].astype(str))

# agg demand

format demand and create zone

In [113]:
print('*** Demand Aggregation ***')

*** Demand Aggregation ***


In [114]:
demand['origin'] = [*zip(demand['lon_ori'],demand['lat_ori'])]
demand['destination'] = [*zip(demand['lon_des'],demand['lat_des'])]
demand = demand.drop(columns=['lat_ori','lon_ori','lat_des','lon_des'])

In [115]:
# get unique zones
zones_set = set(demand['origin'].values).union(set(demand['destination'].values))
zones_dict = {val:'zone_'+str(i) for i,val in enumerate(zones_set)}
zones_df = [{'index':zone,'geometry':Point(val)} for val,zone in zones_dict.items()]

In [116]:
demand['origin'] = demand['origin'].apply(lambda x: zones_dict.get(x))
demand['destination'] = demand['destination'].apply(lambda x: zones_dict.get(x))
zones = gpd.GeoDataFrame(zones_df,crs=4326).set_index('index')

Clustering

In [117]:
agg_dist = float(argv['params']['aggregation']['distance'])
epsg = get_epsg(zones.iloc[0]['geometry'].y, zones.iloc[0]['geometry'].x)
label = agglomerative_clustering(zones.to_crs(epsg), distance_threshold = agg_dist)
zones['cluster'] = label

In [118]:
print(len(zones),'zones agg to',len(zones['cluster'].unique()),' zones')

1163 zones agg to 434  zones


In [119]:
# create a dict zone:cluster
zones['cluster'] = 'zone_' + zones['cluster'].astype(str)
cluster_dict = zones['cluster'].to_dict()

In [120]:
# drop duplicated cluster and rename index with new cluster as zones_id
zones = zones.drop_duplicates('cluster')
zones = zones.set_index('cluster')
zones.index.name='index'

 AGG demand on new zones (volime is sum)

In [121]:
# apply new cluster fict to the demand
demand['origin'] = demand['origin'].apply(lambda x: cluster_dict.get(x))
demand['destination'] = demand['destination'].apply(lambda x: cluster_dict.get(x))

In [122]:
demand = demand.groupby(['origin','destination']).agg(sum).reset_index()
demand.index.name = 'index'

# export zones and demand in outputs

In [123]:
#zones.to_file(output_folder + 'centroids.geojson',driver='GeoJSON')
print('*** Creating and exporting zones ***')

*** Creating and exporting zones ***


In [124]:
# Group the DataFrame by "origin" and "destination", and then convert to JSON
grouped = demand.groupby(["origin", "destination"])["volume"].sum().reset_index()
json_data = {}
data = {}
for _, row in grouped.iterrows():
    origin = row["origin"]
    destination = row["destination"]
    volume = row["volume"]
    if origin not in data:
        data[origin] = {}
    data[origin][destination] = volume

    
json_data['outgoing volume'] = data

In [125]:
# Group the DataFrame by "origin" and "destination", and then convert to JSON
grouped = demand.groupby(["destination", "origin"])["volume"].sum().reset_index()
data = {}
for _, row in grouped.iterrows():
    origin = row["origin"]
    destination = row["destination"]
    volume = row["volume"]
    if destination not in data:
        data[destination] = {}
    data[destination][origin] = volume

json_data['ingoing volume'] = data

In [126]:
    
with open(output_folder + 'zones.json', 'w') as json_file:
    json.dump(json_data,json_file)
with open(output_folder + 'demand.json', 'w') as json_file:
    json.dump(json_data,json_file)

In [127]:
production = demand.groupby(["origin"])["volume"].sum().to_dict()
attraction = demand.groupby(["destination"])["volume"].sum().to_dict()
zones['production'] = zones.index.map(production.get)
zones['attraction'] = zones.index.map(attraction.get)
zones = zones.fillna(0)

In [128]:
convex_hull = zones.unary_union.convex_hull.buffer(1e-3)
voronoi = voronoi_diagram_dataframes(zones['geometry'])
voronoi = gpd.GeoDataFrame(voronoi[0],crs=4326)
voronoi = pd.merge(voronoi,zones[['production','attraction']],left_index=True,right_index=True)
voronoi = voronoi.clip(convex_hull)
voronoi.to_file(output_folder + 'zones.geojson',driver='GeoJSON')
zones.to_file(output_folder + 'demand.geojson',driver='GeoJSON')

/home/boivin/.local/share/virtualenvs/quetzal-77-onnKO/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [129]:
'''
def create_zones_from_nodes(nodes,num_zones=200):
    nodes['x'] = nodes['geometry'].apply(lambda p:p.x)
    nodes['y'] = nodes['geometry'].apply(lambda p:p.y)
    cluster = KMeans(n_clusters=num_zones,random_state=0,n_init='auto')
    cluster.fit(nodes[['x','y']].values)
    geom = [Point(val) for val in cluster.cluster_centers_]
    zones = gpd.GeoDataFrame(range(len(geom)),geometry=geom,crs=4326).drop(columns=0)
    zones.index = 'zone_' + zones.index.astype(str)
    return zones
#create_zones_from_nodes(zones).plot()
'''

"\ndef create_zones_from_nodes(nodes,num_zones=200):\n    nodes['x'] = nodes['geometry'].apply(lambda p:p.x)\n    nodes['y'] = nodes['geometry'].apply(lambda p:p.y)\n    cluster = KMeans(n_clusters=num_zones,random_state=0,n_init='auto')\n    cluster.fit(nodes[['x','y']].values)\n    geom = [Point(val) for val in cluster.cluster_centers_]\n    zones = gpd.GeoDataFrame(range(len(geom)),geometry=geom,crs=4326).drop(columns=0)\n    zones.index = 'zone_' + zones.index.astype(str)\n    return zones\n#create_zones_from_nodes(zones).plot()\n"

# split oneway quenedi links

In [130]:
print('*** Processing cycleways ***')

*** Processing cycleways ***


In [131]:
# split quenedi road links
self = RoadModel(links,nodes,zones,ff_time_col='time')

In [132]:

print('split rlinks to oneways')
self.split_quenedi_rlinks()
#self.zones_nearest_node()

split rlinks to oneways


In [133]:
links = self.road_links
del self

# tag cycleways

In [134]:
index = [idx for idx in links.index if not idx.endswith('_r')]
index_r = [idx for idx in links.index if idx.endswith('_r')]
assert len(index)+len(index_r) == len(links)

In [135]:
# inverted links cycleway is the reverse one
links.loc[index_r,'cycleway'] = links.loc[index_r,'cycleway_reverse'] 

KeyError: 'cycleway_reverse'

split oneway road with cycleway on both side or only in reverse.

In [ ]:
def split_quenedi_cycleway(links, oneway='1',no_col='No'):
    # add oneway links with cycleway_reverse as links. only the one with cycleway_reverse != No.
    # so. you can cycle on oneway with no cycleway only in the oneway direction.
    if 'oneway' not in links.columns:
        print('no column oneway. do not split')
        return
    links_c = links[links['oneway']==oneway].copy()
    if len(links_c) == 0:
        print('all oneway, nothing to split')
    
    links_c = links_c[links_c['cycleway_reverse'] != no_col]
    links_c['cycleway'] = links_c['cycleway_reverse']
    links_c.index = links_c.index.astype(str) + '_c'
    
    # reverse links (a=>b, b=>a)
    links_c = links_c.rename(columns={'a': 'b', 'b': 'a'})
    links_c['geometry'] = links_c['geometry'].apply(lambda g: _reverse_geom(g))
    links = pd.concat([links, links_c])
    return links

In [ ]:
links = split_quenedi_cycleway(links, oneway='1', no_col='no')

In [ ]:
index_c = [idx for idx in links.index if idx.endswith('_c')]
assert len(index)+len(index_r)+len(index_c) == len(links)

In [ ]:
r_cols = [col for col in links.columns if col.endswith('_r')]
if 'cycleway_reverse' in links.columns:
    r_cols = r_cols + ['cycleway_reverse']
links = links.drop(columns=r_cols)

In [ ]:
links['isroad'] = True
ls = links['cycleway'].isin(['yes','shared'])
links.loc[ls,'isroad'] = False


# inclines

In [ ]:
links['incline'] = links['incline'].astype(float)
links['incline'] = links['incline'].fillna(0)


In [ ]:
# reverse incline on reversed links
links.loc[index_r,'incline'] = -links.loc[index_r,'incline']
#create this column for vizualisation
links['incline (abs)'] = abs(links['incline'])

# apply Weight

In [ ]:
# incline weigth (starting point)
default_speed = 20 #kmh
links['eff_speed'] = default_speed   * (1 - cst_incline * np.sin(np.deg2rad(links['incline'])))


In [ ]:
# derate it with road weight
links['cst_road'] = links['highway'].apply(lambda x: cst_road.get(x,1))
links.loc[links['isroad'],'eff_speed'] = links.loc[links['isroad'],'eff_speed'] * links.loc[links['isroad'],'cst_road']

In [ ]:
# derate it with shared cycleway weight
links['cst_shared'] = links['highway'].apply(lambda x: cst_shared.get(x,1))

ls = links.loc[links['cycleway']=='shared'].index
links.loc[ls,'eff_speed'] = links.loc[ls,'eff_speed'] * links.loc[ls,'cst_shared']


transform effective speed to a weight

In [ ]:
# anything with neg speed round to 0. (dijkstra need positive weight)
links['eff_speed'] = links['eff_speed'].apply(lambda x : max(x,0))

In [ ]:
links['weight'] = links['length']/(links['eff_speed']*1000/3600)

In [ ]:
links = links.drop(columns=['cst_road','cst_shared'])

# zone to nodes

get the list of nodes with weight != inf. we do not want to route from or to nodes that are not cycle (ex motorway)

In [ ]:
print('*** Zones to nodes ***')

In [ ]:
tlinks = links[np.isfinite(links['weight'])][['a','b']]
nodes_set = set(tlinks['a']).union(set(tlinks['b']))
possible_nodes = nodes.loc[list(nodes_set)].sort_index()
del tlinks,nodes_set

In [ ]:
zones = zones_nearest_node(zones, possible_nodes)

# Shortest path + volume assignment

In [ ]:
print('*** Assigning volume ***')

In [ ]:
zones_nodes_dict = zones['node_index'].to_dict()
demand['o_zone'] = demand['origin']
demand['d_zone'] = demand['destination']
demand['origin'] = demand['origin'].apply(lambda x: zones_nodes_dict.get(x))
demand['destination'] = demand['destination'].apply(lambda x: zones_nodes_dict.get(x))

In [ ]:
df = links[['a', 'b','weight']].copy()

In [ ]:
df, dropped = drop_duplicated_links(df,sort_column='weight',ascending=False)

In [ ]:
edges = df[['a', 'b', 'weight']].values  # to build the index once and for all
index = build_index(edges)
reversed_index = {v: k for k, v in index.items()}
# apply sparse index on zones
demand, zones_indices = get_zone_index(df, demand, index)

In [ ]:
# apply sparse index on links
df['sparse_a'] = df['a'].apply(lambda x: index.get(x))
df['sparse_b'] = df['b'].apply(lambda x: index.get(x))
volumes_sparse_keys = list(zip(df['sparse_a'],df['sparse_b']))

odv = demand[['o', 'd', 'volume']].values

In [ ]:
edges = df[['a', 'b', 'weight']].values
sparse, _ = sparse_matrix(edges, index=index)
time_matrix, predecessors = parallel_dijkstra(sparse,
                                              directed=True,
                                              indices=zones_indices,
                                              return_predecessors=True,
                                              num_core=num_cores,
                                              keep_running=True)

# this give OD_time/time_matrix on each links. then X links time for the ratio links_time/tot_time


In [ ]:
ab_volumes = assign_volume(odv,predecessors,volumes_sparse_keys,reversed_index)

restrict links to df.index (dropped duplicated links). if not. volume will not be assign on the right link (when duplicated) or, maybe on both.

In [ ]:
links['volume'] = 0
links.loc[df.index,'volume'] = links.loc[df.index].set_index(['a', 'b']).index.map(ab_volumes.get)

In [ ]:
links = links.drop(columns=['isroad'])

In [ ]:
#volume(1/eff_speed - 1/default_speed)
links['time_loss'] = links['volume']* (1/links['eff_speed'] - 1/default_speed)
links.loc[~np.isfinite(links['time_loss']),'time_loss'] = np.nan

# exporting loaded Links

In [ ]:
print('*** Exporting loaded Links ***')
links[['highway','cycleway', 'incline (abs)', 'eff_speed','weight','volume','time_loss','geometry']].to_file(output_folder + 'loaded_links.geojson',driver='GeoJSON')

# links used

In [ ]:
links_list=[]
if 'selectLink' in links.columns:
    links_list += list(links[links['selectLink']=='yes'].index)
if 'selectLink_reverse' in links.columns:
    links_list += list(links[links['selectLink_reverse']=='yes'].index)
if len(links_list)>500:
    print('too many selectLink. keep first 500.')
    links_list = links_list[:500]
    
if len(links_list) > 0:
    links_dict = links.reset_index().set_index(['a','b'])['index'].to_dict()
    json_data={}
    for link in links_list:
        volumes={}
        for o,d,v in odv:
            path = get_path(predecessors,o,d)
            path = [*map(reversed_index.get, path)]
            path = list(zip(path[:-1], path[1:]))
            path = [*map(links_dict.get,path)]
            if link in path:
                for key in path:
                    try:
                        volumes[key] += v
                    except KeyError:
                        volumes[key] = v
        if len(volumes)>0:
            json_data[link] = volumes


    index_set=set()
    for key,item in json_data.items():
        index_set.update(item.keys())

    route = links.loc[list(index_set)][['geometry']]
    route.to_file(output_folder + 'selectLink.geojson',driver='GeoJSON')

    with open(output_folder + 'selectLink.json', 'w') as json_file:
        json.dump({'volume':json_data},json_file)

# OD test

In [ ]:
print('*** OD shortest path ***')

In [ ]:
if od_file_provided:
    od_test['geometry_o'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][0]))
    od_test['geometry_d'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][1]))

In [ ]:
if od_file_provided:
    # find nearest node with KNN. nodes are now the origin and destination.
    od_test['geometry'] = od_test['geometry_o']
    neigh = nearest(od_test, possible_nodes, n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    od_test['origin'] = od_test.index.map(zone_node_dict.get)

    od_test['geometry'] = od_test['geometry_d']
    neigh = nearest(od_test, possible_nodes, n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    od_test['destination'] = od_test.index.map(zone_node_dict.get)

    od_test = od_test.drop(columns=['geometry_o','geometry_d','geometry'])

In [ ]:
if od_file_provided:
    o_nodes = od_test['origin'].values
    d_nodes = od_test['destination'].values

    time_mat, predecessors, node_index = simple_routing(o_nodes,d_nodes,links,weight_col='weight',return_predecessors=True)
    reversed_index = {v: k for k, v in node_index.items()}

    links_dict = links.reset_index().set_index(['a','b'])['index'].to_dict()

    routes = gpd.GeoDataFrame()
    for o,d in  enumerate(od_test['destination'].values):
        path = get_path(predecessors, o, node_index[d])
        path = list(zip(path[:-1], path[1:]))

        path = [(reversed_index[k[0]], reversed_index[k[1]]) for k in path]
        path = [*map(links_dict.get,path)]

        route = links.loc[path]
        route['od_name'] =  od_test.iloc[o]['name']
        routes = pd.concat([routes,route])

# export OD routes

In [ ]:
if od_file_provided:
    routes.to_file(output_folder + 'od_routes.geojson',driver='GeoJSON')

#  kpis

In [ ]:
print('*** KPI ***')

network

In [ ]:
colors=["#559BB4","#91A564","#DC9100","#D22328","#8C4B7D"]


In [ ]:
links['volkm'] = links['volume']*links['length']/1000
res = links.groupby('cycleway')[['volkm']].agg(sum)
#res['volkm'].plot.pie()
res = res.T.fillna(0)
for col in ['no','yes','shared']:
    if col not in res.columns:
        res[col]=0
        
res.columns = res.columns+' (vol.km)'
res = res.rename(index={'volkm':''})
res.columns.name=''
res['cycle (vol.km)'] = res.sum(axis=1)

res['no (%vol)'] = 100* res['no (vol.km)'] / res['cycle (vol.km)']
res['shared (%vol)'] = 100* res['shared (vol.km)'] / res['cycle (vol.km)']
res['yes (%vol)'] = 100* res['yes (vol.km)'] / res['cycle (vol.km)']

In [ ]:
# crow fligh
geom_dict = add_geometry_coordinates(zones)[['y_geometry','x_geometry']].apply(tuple,axis=1).to_dict()
demand['distance'] = demand[['o_zone','d_zone']].apply(lambda x: get_flight_distance([*map(geom_dict.get, x.values)]), axis=1)
res['crow (vol.km)'] = sum(demand['volume'] * demand['distance'])/1000

In [ ]:
res['cycle-crow (vol.km)'] = res['cycle (vol.km)'] - res['crow (vol.km)']
res['cycle/crow (%)'] = 100 * res['cycle (vol.km)']/ res['crow (vol.km)']

In [ ]:
# percent of links with cycleway (no volume here)
tmp = links.groupby('cycleway')[['length']].agg(sum)/1000
tmp = tmp.T.fillna(0)
for col in ['no','yes','shared']:
    if col not in tmp.columns:
        tmp[col]=0
tmp.columns = tmp.columns+' (%)'
tmp = tmp.rename(index={'length':''})
tmp.columns.name=''
tot = tmp.sum(axis=1)

tmp['no (%)'] = 100* tmp['no (%)'] / tot
tmp['shared (%)'] = 100* tmp['shared (%)'] / tot
tmp['yes (%)'] = 100* tmp['yes (%)'] / tot

In [ ]:
res = pd.concat([res,tmp],axis=1)

In [ ]:
res

In [ ]:
res.to_csv(output_folder+'network_kpi.csv')

In [ ]:
# PLOT
data = put_units_in_row(res)
plot = render_mpl_table(data, header_size=12, font_size=12, row_height=0.4, col_width=1.3, index_width_ratio=1)
title = 'Network table'
plot.set_title(title , fontsize=12)
file = output_folder+'6_'+'{title}.png'.format(title=normalize(title))
plot.get_figure().savefig(file, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
res['crow (vol.km)'].plot(kind='barh',ax=ax, color=colors[0], position=0,width=0.4)
res[['yes (vol.km)', 'shared (vol.km)', 'no (vol.km)']].plot(kind='barh',stacked=True, ax=ax,position=1,width=0.4,color=colors[1:])

spacing = (ax.get_xticks()[1] - ax.get_xticks()[0])/2
annot_position = res.iloc[0]['cycle (vol.km)'] + spacing/2
plt.ylim([-0.5,len(res)-0.5])
plt.xlim(ax.get_xticks()[0], annot_position+spacing)
plt.annotate('+' + str(round(res.iloc[0]['cycle/crow (%)'])-100) + '%', xy=(annot_position,-0.18), ha='center', va='bottom',fontsize=12)

plt.legend(['as the crow flies', 'cycleway','shared cycleway', 'road'],loc="lower right", ncol=1)
plt.gca().invert_yaxis()
plt.xlabel("Distance (vol.km)")
plt.grid(True, 'major', linestyle='-', axis='both')
ax.set_axisbelow(True)
#plt.gcf().axes[0].xaxis.get_major_formatter().set_scientific(False)
ax.xaxis.set_major_formatter(lambda x,y: '{:,}'.format(int(x)).replace(',', ' '))

plt.title('Total cycle distance vs as the crow flies distance')
plt.savefig(output_folder+'1_total_distance_km.png')

In [ ]:
ax = res[['yes (%vol)', 'shared (%vol)', 'no (%vol)']].plot(kind='barh',stacked=True,figsize=(10, 4),color=colors[1:])
plt.legend(loc="upper right", ncol=1)
plt.gca().invert_yaxis()
plt.xlabel("Distance (% of vol.km)")
plt.grid(True, 'major', linestyle='-', axis='both')
ax.set_axisbelow(True)
plt.legend([ 'cycleway','shared cycleway', 'road'],loc="upper right", ncol=1)
plt.title('Total percent of cycle distance')
plt.savefig(output_folder+'2_total_distance_percent.png')

In [ ]:
ax = res[['yes (%)', 'shared (%)', 'no (%)']].plot(kind='barh',stacked=True,figsize=(10, 4),color=colors[1:])
plt.legend(loc="upper right", ncol=1)
plt.gca().invert_yaxis()
plt.xlabel("Distance (%)")
plt.grid(True, 'major', linestyle='-', axis='both')
ax.set_axisbelow(True)
plt.legend([ 'cycleway','shared cycleway', 'road'],loc="upper right", ncol=1)
plt.title('Percent of cycleway on the network')
plt.savefig(output_folder+'3_percent_cycleway_network.png')

# KPI  per OD

In [ ]:
if od_file_provided:
    res = routes.groupby(['od_name','cycleway'])[['length']].agg(sum)/1000

    res = res.unstack().fillna(0)
    res.columns = res.columns.levels[1]
    for col in ['no','yes','shared']:
        if col not in res.columns:
            res[col]=0
    res.columns = res.columns+' (km)'

    res['cycle (km)'] = res.sum(axis=1)

    res['no (%)'] = 100* res['no (km)'] / res['cycle (km)']
    res['shared (%)'] = 100* res['shared (km)'] / res['cycle (km)']
    res['yes (%)'] = 100* res['yes (km)'] / res['cycle (km)']

In [ ]:
if od_file_provided:
    geom_dict = add_geometry_coordinates(nodes)[['y_geometry','x_geometry']].apply(tuple,axis=1).to_dict()
    od_test['crow (km)'] = od_test[['origin','destination']].apply(lambda x: get_flight_distance([*map(geom_dict.get, x.values)]), axis=1)/1000
    tmp_dict = od_test.set_index('name')['crow (km)'].to_dict()
    res['crow (km)'] = res.index.map(tmp_dict.get)

In [ ]:
if od_file_provided:
    res['cycle-crow (km)'] = res['cycle (km)'] - res['crow (km)']
    res['cycle/crow (%)'] = 100 * res['cycle (km)']/ res['crow (km)']

In [ ]:
if od_file_provided:
    res.to_csv(output_folder+'od_kpi.csv')

In [ ]:
if od_file_provided:
    # PLOT
    data = put_units_in_row(res)
    plot = render_mpl_table(data, header_size=12, font_size=12, row_height=0.4, col_width=1.5, index_width_ratio=2.2)
    title = 'OD table'
    plot.set_title(title , fontsize=12)
    file = output_folder+'7_'+'{title}.png'.format(title=normalize(title))
    plot.get_figure().savefig(file, bbox_inches='tight')

In [ ]:
if od_file_provided:
    fig, ax = plt.subplots(figsize=(10,10))
    res['crow (km)'].plot(kind='barh',ax=ax, color=colors[0], position=0, width=0.4)
    res[['yes (km)', 'shared (km)', 'no (km)']].plot(kind='barh', stacked=True, ax=ax, position=1, width=0.4, color=colors[1:])


    spacing = (ax.get_xticks()[1] - ax.get_xticks()[0])/2
    annot_position = res['cycle (km)'].values + spacing/2
    plt.ylim([-0.5,len(res)-0.5])
    plt.xlim(ax.get_xticks()[0], max(annot_position) + spacing)
    annot_percent = ['+'+str(round(val)-100)+'%' for val in res['cycle/crow (%)'].values]
    for i in range(len(annot_position)):
        plt.annotate(annot_percent[i], xy=(annot_position[i], i-0.1), ha='center', va='bottom',fontsize=10)



    plt.legend(['as the crow flies', 'cycleway','shared cycleway', 'road'],loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
    plt.gca().invert_yaxis()
    plt.xlabel("Distance (km)")
    plt.ylabel('Origin-Destination name')
    plt.grid(True, 'major', linestyle='-', axis='both')
    ax.set_axisbelow(True)



    plt.title('Cycle distance vs as the crow flies distance')
    plt.savefig(output_folder+'4_od_distance_km.png')

In [ ]:
if od_file_provided:
    ax = res[['yes (%)', 'shared (%)', 'no (%)']].plot(kind='barh',stacked=True,figsize=(10, 10),color=colors[1:])
    plt.legend(loc="upper right", ncol=1)
    plt.gca().invert_yaxis()
    plt.xlabel("Distance (%)")
    plt.grid(True, 'major', linestyle='-', axis='both')
    ax.set_axisbelow(True)
    plt.legend([ 'cycleway','shared cycleway', 'road'],loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
    plt.title('percent of distance on cycleway')
    plt.savefig(output_folder+'5_od_distance_percent.png')

In [ ]:
print('*** Finish! ***')

# end

# merging

In [ ]:
'''
def merge_quenedi_rlinks(road_links,new_col):
    if 'oneway' not in road_links.columns:
        print('no column oneway. do not merge')
        return
    #get reversed links
    index_r = [idx for idx in road_links.index if idx.endswith('_r')]
    if len(index_r) == 0:
        print('all oneway, nothing to merge')
        return
    links_r = road_links.loc[index_r].copy()
    # create new reversed column with here speed and time
    links_r[new_col + '_r'] = links_r[new_col]
    # reindex with initial non _r index to merge
    links_r.index = links_r.index.map(lambda x: x[:-2])
    links_r = links_r[[new_col + '_r']]
    # drop added _r links, merge new here columns to inital two way links.
    road_links = road_links.drop(index_r, axis=0)
    # drop column if they exist before merge. dont want duplicates
    if new_col + '_r' in road_links.columns:
        road_links = road_links.drop(columns=new_col + '_r')
    road_links = pd.merge(road_links, links_r, left_index=True, right_index
                                =True, how='left')
    return road_links
'''

In [ ]:
#links = merge_quenedi_rlinks(links,'volume')

# test